In [8]:
import spacy
from tqdm import tqdm  # Affichage d'une barre de progression
import difflib  # Recherche de similarité entre deux chaînes de caractères
import pandas as pd
import matplotlib.pyplot as plt
from spacy.language import Language
from spacy_langdetect import LanguageDetector

# Charger le modèle entraîné pour l'itinéraire
nlp_itinerary = spacy.load("../../models/saved_models/model-best/")
# Charger le modèle pré-entraîné en français
nlp_fr = spacy.load("fr_core_news_md")


# -------------------------------------------------------------------------------
# Fonction pour installer et configurer le détecteur de langue
# -------------------------------------------------------------------------------
def install_language_detector():
    # Vérifier si la factory 'language_detector' existe déjà
    if 'language_detector' not in nlp_fr.pipe_names:
        # Si elle n'existe pas, définir la factory
        if not hasattr(Language.factories, 'language_detector'):
            @Language.factory('language_detector')
            def language_detector(nlp, name):
                return LanguageDetector()
        # Ajouter le détecteur de langue à la pipeline
        nlp_fr.add_pipe('language_detector', last=True)
        print("LanguageDetector ajouté à la pipeline")
    else:
        print("LanguageDetector est déjà présent dans la pipeline")


# Installer et configurer le détecteur de langue
install_language_detector()




# -------------------------------------------------------------------------------
# Fonction pour extraire les entités de départ et destination du texte
# -------------------------------------------------------------------------------
def extract_locations(text, lang_model, itinerary_model, verbose=False):
    doc_lang = lang_model(text)
    doc_itinerary = itinerary_model(text)

    if verbose:
        print("Entités détectées avec le modèle linguistique :")
        spacy.displacy.render(doc_lang, style='ent', jupyter=True)
        print("Entités détectées avec le modèle itinéraire :")
        spacy.displacy.render(doc_itinerary, style='ent', jupyter=True)

    departure, destination = None, None

    # Vérifier si le texte est bien en français
    if doc_lang._.language['language'] != 'fr':
        return "NON_FRANÇAIS"

    # Parcourir les entités détectées par le modèle d'itinéraire
    for ent in doc_itinerary.ents:
        if ent.label_ == "DEPARTURE":
            departure = ent.text
        elif ent.label_ == "DESTINATION":
            destination = ent.text

    if not departure or not destination:
        return "PAS_DE_TRAJET"
    
    return {"départ": departure, "destination": destination}


# -------------------------------------------------------------------------------
# Fonction pour traiter un fichier et extraire les informations de départ et destination
# -------------------------------------------------------------------------------
def process_file(filepath, lang_model, itinerary_model, verbose=False):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    result_trips = []
    for line in lines:
        elements = line.strip().split(',', 1)
        id_ = elements[0] if elements[0].isdigit() else 0
        text = elements[1] if len(elements) > 1 else elements[0]

        trips = extract_locations(text, lang_model, itinerary_model, verbose)
        if trips == "NON_FRANÇAIS" or trips == "PAS_DE_TRAJET":
            result_trips.append(f"{id_},{trips}")
        else:
            result_trips.append(f"{id_},{trips['départ']},{trips['destination']}")

    if verbose:
        print("Trajets extraits :")
        print(result_trips)

    return result_trips


# -------------------------------------------------------------------------------
# Fonction de comparaison des résultats extraits avec les résultats attendus
# -------------------------------------------------------------------------------
def compare_trips(extracted, expected):
    errors = 0
    for i in range(len(extracted)):
        if extracted[i] != expected[i]:
            errors += 1
            print(f"Erreur ligne {i+1} :")
            print(f"Extrait : {extracted[i]}")
            print(f"Attendu : {expected[i]}")

    # Calcul du taux de réussite
    success_rate = round((len(extracted) - errors) / len(extracted) * 100, 2)
    print(f"\nTaux de réussite : {success_rate}% ({errors} erreurs sur {len(extracted)} lignes).\n")

    # Affichage d'un graphique circulaire pour visualiser le taux de réussite
    labels = 'Réussites', 'Erreurs'
    sizes = [len(extracted) - errors, errors]
    colors = ['green', 'red']
    plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')
    plt.show()

    # Comparaison des résultats sous forme de tableau avec pandas
    df = pd.DataFrame({
        'ID': [item.split(',')[0] for item in extracted],
        'Départ Attendu': [item.split(',')[1] if len(item.split(',')) > 1 else '' for item in expected],
        'Départ Prévu': [item.split(',')[1] if len(item.split(',')) > 1 else '' for item in extracted],
        'Destination Attendue': [item.split(',')[2] if len(item.split(',')) > 2 else '' for item in expected],
        'Destination Prévue': [item.split(',')[2] if len(item.split(',')) > 2 else '' for item in extracted],
    })

    print("\nComparaison des résultats :\n", df)


# -------------------------------------------------------------------------------
# Fonction de correction des villes en utilisant la liste de villes
# -------------------------------------------------------------------------------
def correct_cities(trips, city_list, verbose=False):
    corrected_trips = []
    for trip in trips:
        parts = trip.upper().replace('É', 'E').replace('-', ' ').split(',')
        corrected_trip = ','.join([difflib.get_close_matches(item, city_list, n=1, cutoff=0.8)[0] if difflib.get_close_matches(item, city_list, n=1, cutoff=0.8) else item for item in parts])
        corrected_trips.append(corrected_trip)

        if verbose:
            print(f"Avant : {trip}")
            print(f"Après correction : {corrected_trip}")

    return corrected_trips


# -------------------------------------------------------------------------------
# Fonction principale pour l'analyse d'un fichier et la comparaison avec les résultats attendus
# -------------------------------------------------------------------------------
def analyze_nlp(file_path_input, file_path_output):
    with open(file_path_input, 'r') as file:
        print(f"Lecture du fichier d'entrée :\n{file.read()}")

    with open(file_path_output, 'r', encoding='utf-8') as file:
        reference_trips = [line.strip().rstrip(',') for line in file.readlines()]

    print("\nANALYSE BRUTE (SANS PRÉTRAITEMENT)\n")
    raw_output_trips = process_file(file_path_input, nlp_fr, nlp_itinerary, True)
    compare_trips(raw_output_trips, reference_trips)

    print("\nANALYSE APRÈS PRÉTRAITEMENT\n")
    processed_output_trips = correct_cities(raw_output_trips, '../utils/extra_datas/urban_geodata_basic_v1.0.txt')
    processed_reference_trips = correct_cities(reference_trips, '../utils/extra_datas/urban_geodata_basic_v1.0.txt')
    compare_trips(processed_output_trips, processed_reference_trips)


# -------------------------------------------------------------------------------
# Exécution de l'analyse avec les fichiers d'entrée et de sortie
# -------------------------------------------------------------------------------
file_path_input = '../utils/test_data/nlp_input_data.txt'
file_path_output = '../utils/test_data/nlp_output_data.txt'

analyze_nlp(file_path_input, file_path_output)


ValueError: [E004] Can't set up pipeline component: a factory for 'language_detector' already exists. Existing factory: <function install_language_detector.<locals>.language_detector at 0x00000221669471A0>. New factory: <function install_language_detector.<locals>.language_detector at 0x0000022166927E20>